In [5]:
import pandas as pd
import os
import sys
import pickle
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy
import psycopg2

In [6]:
STOP_WORDS = { "the",
    "and", "or", "the", "a", "an", "to", "of", "in", "on", "for", "with",
    "by", "at", "from", "as", "is", "are", "this", "that", 'tv', 'collection',
    'episodes' ,'episode', 'и', 'ну', 'не', 'ссср','shorts', 'y', 'kidsmania',
    'un', 'el', 'de', 'be', 'episodio'
}

In [7]:
def tokenize_clean(text):
    tokens = re.findall(r"\b[^\W_]+\b", text.lower(), flags=re.UNICODE)
    return [ t.strip() for t in tokens if not t.isdigit() and t not in STOP_WORDS ]

In [8]:
#### get new reels

In [9]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))
conn = db.connect()
req1 = ''' select max(data_date) from intl_yt_reels_stat iyrs  '''
last_upload = pd.read_sql(text(req1), conn)
ref_date = last_upload['max'].to_list()[0] - datetime.timedelta(days=10)
req2 = ''' select iyr.yt_reel_id, reel_name, yt_ch_name, cartoon, brand_id, product_id,language from intl_yt_reels iyr 
join intl_yt_channels2reels iyrcr 
on iyr.yt_reel_id = iyrcr.yt_reel_id 
join intl_yt_channels iyc 
on iyrcr.yt_ch_id = iyc.yt_ch_id 
where iyc."language" in ('США','Английский', 'Испанский', 'Арабский', 'Португальский','Китайский')
and iyr.removed = 0  
and  "date">= '{}' '''.format(ref_date.strftime( format = "%Y-%m-%d")) # and brand_id='' 
df_in = pd.read_sql(text(req2), conn)
conn.close()

In [10]:
df_in['name'] = df_in.yt_ch_name + ' '+df_in.reel_name
df_in['tokens'] = df_in['name'].apply(tokenize_clean)
df_in['bigrams'] = df_in['tokens'].apply(lambda x: list(zip(x, x[1:])))

In [11]:
with open("final_grams_set.pkl", 'rb') as file:
    final_grams_set = pickle.load(file)

In [12]:
df_in['detected'] = 0 

In [13]:
for gram in final_grams_set:
    if type(gram) == str:
        print('str:', gram)
        df_in.loc[:,'detected'] += df_in['tokens'].apply(lambda cell: int(gram in cell))
    else:
        print('touple:', gram)
        df_in.loc[:,'detected'] += df_in['bigrams'].apply(lambda cell: int(gram in cell))

touple: ('animales', 'guauguau')
touple: ('birdies', 'learning')
touple: ('pilotos', 'aventuras')
touple: ('new', 'caroons')
touple: ('planes', 'learn')
touple: ('minute', 'cartoon')
touple: ('عربي', 'عائلة')
str: competiciones
touple: ('new', 'record')
touple: ('tímido', 'valiente')
touple: ('race', 'say')
str: caleidoscopio
str: mosaico
touple: ('laranja', 'sintamse')
touple: ('odd', 'numbers')
touple: ('trouble', 'nursery')
touple: ('fin', 'del')
touple: ('get', 'set')
touple: ('عائلةأريكوت', 'رسوم')
touple: ('animal', 'learn')
touple: ('guests', 'gone')
touple: ('aventuras', 'casitas')
touple: ('carta', 'nuevos')
touple: ('juegos', 'deportivos')
touple: ('الساحر', 'كرتون')
touple: ('الأول', 'ألحان')
touple: ('inside', 'outside')
touple: ('beging', 'end')
str: umka
touple: ('spring', 'miracle')
touple: ('música', 'con')
touple: ('mania', 'عائلة')
touple: ('kids', 'orange')
touple: ('خيرا', 'تحصد')
touple: ('números', 'plastilina')
touple: ('o', 'elástico')
touple: ('world', 'game')


In [14]:
df_in[(df_in.detected>0) & (df_in.brand_id == '')].to_excel('markuped_new.xlsx', index = False)

In [11]:
df_in.to_excel('markuped_test.xlsx', index = False)